Rag example usage with Pydantic AI & ChromaDB

In [ ]:
%pip install fastapi
%pip install pydantic_ai
%pip install chromadb
%pip install python_dotenv

In [ ]:
import json
import os
import uuid
from dataclasses import dataclass
from python_dotenv import load_dotenv

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import nest_asyncio
import requests
from chromadb.api.models.Collection import Collection
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider

In [32]:
# Init VectorStore

def build_vector_store():
    url = "https://raw.githubusercontent.com/syv-ai/it-center-fyn/main/datasets/teknologier.json"
    response = requests.get(url)
    data = response.json()
   
    client = chromadb.Client()

    # Option 1: Use SentenceTransformer directly (no external API needed)
    ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="intfloat/multilingual-e5-large"
    )

    ## Option 2: Use external OpenAI-compatible embedding endpoint
    # ef = embedding_functions.OpenAIEmbeddingFunction(
    #     api_key="None",
    #     model_name="intfloat/multilingual-e5-large",
    #     api_base="http://localhost:8000/v1"   # Would point to external embedding server
    # )
    
    try:
        client.delete_collection(name="knowledge_base") # Deleting collection for demo purposes
    except Exception:
        pass
    collection = client.get_or_create_collection(
        name="knowledge_base",
        embedding_function=ef,
        metadata={"description": "Teknologier database"}
    )
   
    ids = []
    documents = []
    metadatas = []
    
    for item in data:
        doc_id = str(uuid.uuid4())
        ids.append(doc_id)
        documents.append(item["content"])
        metadatas.append({
            "title": item["title"],
            "content": item["content"],
            "source": item["source"]
        })
    print("Generating embeddings...")
    collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas
    )
    
    print(f"Inserted {len(ids)} documents into vector store")
    return collection

collection = build_vector_store()
results = collection.get()
print(json.dumps(results, indent=4))

Generating embeddings...
Inserted 15 documents into vector store
{
    "ids": [
        "d794baa2-0efe-4418-b30e-84b46cc45d55",
        "abd8a12b-b100-46aa-8133-a4a1b7625214",
        "2d4aba26-eaad-4973-84e4-516968a983d1",
        "504f97ce-7834-4885-9ac8-e7c364868e2c",
        "b0279786-3165-4466-93be-7f44faa7b52d",
        "63d2d9a8-d356-4288-bc07-89a21c353fa2",
        "9f3cd0f9-8162-4e29-8203-b7add2589e38",
        "9e24e60b-4533-4cff-a89a-b289687b0570",
        "c79fd81e-08c5-4fcf-81e5-da80ed26e090",
        "6757fd63-bcff-4430-8730-1118d0415278",
        "62644821-f829-48ea-bdf7-8df200ddd6e6",
        "c09acf9a-4e1e-497d-86db-cca5d56e5ae2",
        "e96ed035-798a-4579-991f-919a64e15763",
        "30d57dfa-bf4f-4818-886f-c982e1786f8d",
        "ba85e386-cef6-4852-ae0c-c851fa9f9da1"
    ],
    "embeddings": null,
    "documents": [
        "Python er et h\u00f8jniveau programmeringssprog skabt af Guido van Rossum i 1991. Det er kendt for sin enkle og l\u00e6sbare syntax der minder

In [40]:
load_dotenv()  # Load .env file
model_name="syvai/danskgpt-v2.1"
base_url = "https://api.syv.ai/v1" # OpenAI-compatible API inference endpoint. Ollama, vllm, etc.
api_key = os.getenv("SYVAI_API_KEY") # None required for local inference

provider = OpenAIProvider(base_url=base_url, api_key=api_key)
model = OpenAIChatModel(model_name=model_name, provider=provider)

In [47]:
# Simple usage

agent = Agent(
    model=model,
    system_prompt="""Du er en dansk assistent.
Svar direkte, korrekt og naturligt på dansk.
Brug kun fakta og skriv et fuldt svar.
Inkluder kildehenvisninger i teksten som [1], [2], ... og tilføj en liste over kilder til sidst.
""",
)

query = "Hvad er FastAPI?"

results = collection.query(
    query_texts=[query],
    n_results=3
    )

context = json.dumps(results['metadatas'][0], ensure_ascii=False, indent=2)

query += "\n\nSvar med kilder:\n\n" + context

nest_asyncio.apply()
result = agent.run_sync(query)
print(result.output)

FastAPI er et moderne, hurtigt web framework til at bygge APIs med Python. Det er baseret på standard Python type hints og blev skabt af Sebastián Ramírez. FastAPI er bygget oven på Starlette for web delen og Pydantic for data delen [1].

Nogle af de vigtigste funktioner i FastAPI inkluderer:

1. **Automatisk OpenAPI dokumentation**: FastAPI genererer automatisk OpenAPI dokumentation, der kan tilgås via /docs endpoint med Swagger UI eller /redoc for ReDoc interface [1].

2. **Type hints**: Frameworket bruger Python type annotations til automatisk request validering, serialisering og dokumentation [1].

3. **Asynkron programmering**: FastAPI understøtter asynkron programmering med async/await syntax, hvilket gør det ekstremt performant [1].

4. **Dependency injection**: Dependency injection systemet i FastAPI gør det nemt at dele logik mellem endpoints og håndtere authentication, database connections og andre afhængigheder [1].

5. **Path og query parameters**: Path parameters defineres

In [49]:
# Retrieval as tool

@dataclass
class Deps:
    collection: Collection

agent = Agent(
    model=model,
    system_prompt="""Du er en dansk assistent.
Svar direkte, korrekt og naturligt på dansk.
Brug kun fakta og skriv et fuldt svar.
Inkluder kildehenvisninger i teksten som [1], [2], ... og tilføj en liste over kilder til sidst.
""",
)

@agent.tool
async def retrieve(ctx: RunContext[Deps], search_query: str) -> str:
    """Søg i vores database og returner relevante kilder.

Args:
    ctx: kontekst.
    search_query: Brugerens spørgsmål.
"""

    results = ctx.deps.collection.query(
        query_texts=[search_query],
        n_results=3
    )
    return json.dumps(results['metadatas'][0], ensure_ascii=False, indent=2)

async def run_rag(query: str, history: list):
    deps = Deps(collection=collection)
    result = await agent.run(query, deps=deps, message_history=history)
    return result

In [ ]:
# Chat history management
history = []

query = "Brug retrieve værktøjet til dette spørgsmål: Hvad er Python?" # Explicitly use tool, not needed depending on model & setup
result_1 = await run_rag(query, history)
history = result_1.all_messages()
print("Question\n", query)
print("Answer\n", result_1.output)
print("Usage\n", result_1.usage())

query = "Hvad spurgte jeg om lige før?"
result_2 = await run_rag(query, history)
history = result_2.all_messages()# Update history again
print("\nQuestion\n", query)
print("Answer\n", result_2.output)
print("Usage\n", result_2.usage())

Question
 Hvad er Python?
Answer
 Python er et højniveausprogrammeringssprog, der er kendt for sin læsevenlighed og enkelhed. Det blev oprettet af Guido van Rossum og blev første gang udgivet i 1991. Python understøtter flere programmeringsparadigmer, herunder objektorienteret, imperativ og funktionel programmering. Det er et cross-platform-sprog, hvilket betyder, at det kan køre på forskellige operativsystemer som Windows, macOS og Linux.

Python har en omfattende standardbibliotek, der giver adgang til systemfunktioner og opgaver som filhåndtering, netværksprogrammering og webservices. Derudover er der et stort økosystem af tredjepartsbiblioteker og -værktøjer, der udvider Python's funktioner inden for områder som videnskabelig beregning, dataanalyse, maskinlæring og webudvikling.

Nogle af de mest populære biblioteker og rammer inkluderer:

1. **NumPy**: Et bibliotek til numeriske beregninger og videnskabelig computing.
2. **Pandas**: Et bibliotek til dataanalyse og -manipulation.
3